In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from six.moves import urllib
import zipfile
from scipy import stats

In [ ]:
cyc = pd.read_csv(r'/Users/pedrorodriguez/Desktop/Springboard/Capstone Project/Second Part/Cycle_time_per_station')
dtf = pd.read_csv(r'/Users/pedrorodriguez/Desktop/Springboard/Capstone Project/Second Part/Downtime_freq')
dtt = pd.read_csv(r'/Users/pedrorodriguez/Desktop/Springboard/Capstone Project/Second Part/Downtime_time')

In [ ]:
print('Cycle_time_data shape:         ', cyc.shape, '\nDowntime_times data shape:     ', dtt.shape, '\nDowntime frequency data shape: ', dtf.shape)

## Distribution of Cycle Time

In [ ]:
#For Cycle time
plt.hist(cyc['Cycle_time'])
plt.show()

The distribution is skewed to the right and not normal.

In [ ]:
cyc_sec_mean = cyc['Cycle_time'].mean()
cyc_sec_median = cyc['Cycle_time'].median()
cyc_sec_std =  cyc['Cycle_time'].std()
print( 'The mean for every cycle time is', round(cyc_sec_mean, 2), ',the median is', round(cyc_sec_median , 2) ,'\nand the standard deviation is', round(cyc_sec_std, 2))

In [ ]:
from scipy.stats import t
def margin_of_error(data, value_conf):
   print('Standard Error:  ', round((np.std(data)/np.sqrt(len(data))), 4),
       '\nMargin of Error: ', round(((t.ppf(value_conf, len(data))) * (np.std(data)/np.sqrt(len(data)))), 4),
       '\nIntervals:     ', '(',round(np.mean(data) - ((t.ppf(value_conf, len(data))) * (np.std(data)/np.sqrt(len(data)))), 4) , ',' ,round(((t.ppf(value_conf, len(data))) * (np.std(data)/np.sqrt(len(data)))) + np.mean(data), 4),')')
 

In [ ]:
#Calculate the margin of error, here we created a code to calculate the margin of error
margin_of_error(cyc['Cycle_time'], 0.95)

## Distribution of Downtime Freq

In [ ]:
plt.hist(dtf['Downtime'])
plt.show()

The distribution is skewed to the right and not normal.

In [ ]:
dtf_mean = dtf['Downtime'].mean()
dtf_std = dtf['Downtime'].std()

print('The mean of downtimes ocurred is', round(dtf_mean,2), '\nthe stand. dev. is ', round(dtf_std, 2))

In [ ]:
margin_of_error(dtf['Downtime'], 0.95)

## Distribution of Downtime times

In [ ]:
plt.hist(dtt['value'], bins=50)
plt.show()

The distribution is skewed to the right and not normal.

In [ ]:
dtt_mean = dtt['value'].mean()
dtt_std = dtt['value'].std()
print('The mean of downtime time is', round(dtt_mean, 2), 'and the standard deviation is', round(dtt_std, 2))

In [ ]:
margin_of_error(dtt['value'], 0.95)

### Let's investigate if exist a correlation between features of Cycle_time data and downtime_time data. 

In [ ]:
print(cyc.dtypes)
print('\n',dtt.dtypes)

Lets first prepare both data to merge and perform the correlation study.

In [ ]:
cyc_2 = cyc
cyc_2 = cyc_2.rename(columns=   {'Unnamed: 0' : 'ID', 'station': 'station_no.'})
dtf =   dtf.rename(columns=     {'Unnamed: 0' : 'ID'})
dtt =   dtt.rename(columns=     {'Unnamed: 0' : 'ID'})
dtt =   dtt.rename(columns=     {'value' : 'downtime'})

In [ ]:
cyc_2['station_no.'] = cyc_2['station_no.'].astype(int)
del cyc_2['production_line']
del cyc_2['Sample']

In [ ]:
del dtt['station']
del dtt['variable']

In [ ]:
ct_dt_data = pd.merge(cyc_2, dtt, how = 'left')
ct_dt_data

### Now lets investigate if there is any correlation between features.

In [ ]:
ct_dt_data.info()

In [ ]:
sns.heatmap(ct_dt_data[['Cycle_time', 'downtime']].corr(), annot=True)

In [ ]:
sns.regplot(data= ct_dt_data, x= 'Cycle_time', y= 'downtime')

The heatmap and regression plot shown that there is no correlation between any features. 

### Let's go back and see how the cycle time behave between production line

In [ ]:
sns.boxplot(data= cyc, x= 'production_line', y= 'Cycle_time')

The boxplot shows the production line number 5 and 7 have outlier and the production line 1 and 2 shows a high data dispertion. Lets study the station 5 to see any findings.

In [ ]:
st5_ct = cyc[cyc['production_line'] == 5.0]
st5_ct['station'] = st5_ct['station'].astype(int)

In [ ]:
plt.hist(st5_ct['Cycle_time'])

In [ ]:
sns.barplot(data= st5_ct, x= 'station', y = 'Cycle_time')

As we can see the station 34 have the highest values among all station and shown as an outlier in the histogram. We will have to investigate more about this station and ask the company about this value.

In [ ]:
st5_ct_mean = st5_ct['Cycle_time'].mean()
st5_ct_std = st5_ct['Cycle_time'].std()

print('The average of the cycle time of the production line is', round(st5_ct_mean, 3), 'with a standard deviation of', round(st5_ct_std, 2))

### After talking with someone from the company confirm the station 34 was high and we discovered that every value should be divided by 4 because the machine operate with 4 units at the same time.

In [ ]:
#Let's devide every value of station 34 to fix this issue.
st5_ct['Cycle_time'] = np.where(st5_ct['station'] == 34, st5_ct['Cycle_time']/4, st5_ct['Cycle_time'])
   

In [ ]:
plt.hist(st5_ct['Cycle_time'])

After correcting the data of station 5, we can see that the distribution now looks like skewed to the right and not normal. 

In [ ]:
st5_ct_mean = st5_ct['Cycle_time'].mean()
st5_ct_std = st5_ct['Cycle_time'].std()

print('The average of the cycle time of the production line is', round(st5_ct_mean, 3), 'with a standard deviation of', round(st5_ct_std, 2))

In [ ]:
margin_of_error(st5_ct['Cycle_time'], 0.95)

### Now let's study the Downtime frequence

In [ ]:
#Lets check the Downtime frequence
dtf.head()
del dtf['station']

In [ ]:
sns.lineplot(data= dtf, x='Date', y='Downtime')

In [ ]:
sns.boxplot(data=dtf, x= 'Year', y= 'Downtime')

In [ ]:
dtf_year = dtf.groupby(['Year'])[['Downtime']].mean()
dtf_year.head()

As we can see in the boxplot and line plot, the downtime that occurred in 2019 is higher than in 2018. To be sure about this we are going to calculate the margin of error and see if the mean of 2018 fall inside the 2019 interval.


In [ ]:
dtf_2018 = dtf[dtf['Year'] == 2018][['Downtime']]
dtf_2019 = dtf[dtf['Year'] == 2019][['Downtime']]

In [ ]:
print(margin_of_error(dtf_2018['Downtime'], 0.95))

The error margin of 2018 is 0.13, indicating the intervals are 4.74168 and 4.74168, suggesting that the average of 2019 falls outside the 2018 intervals; this means the difference between 2018 and 2019 averages can be significant. 

In [ ]:
margin_of_error(dtf_2019['Downtime'], 0.95)

The same happens in 2019; the error margin is 0.2 making the interval from 6.77 to 6.77. The average of 2018 does not fall between the interval of 2019.

### To confirm that the average of downtime in 2018 and 2019 is significantly different, we will perform a 2 sample t-test. Assuming that the Null hypothesis means both mean samples are equal and the alternate is not equal. 

In [ ]:
var18 = np.var(dtf_2018["Downtime"])
var19 = np.var(dtf_2019["Downtime"])

print('2018 variance:', round(np.var(dtf_2018["Downtime"]), 2), '\n2019 variance:', round(np.var(dtf_2019["Downtime"]), 2), '\nThe ratio of the larger sample variance to the smaller sample variance is', round(var19/var18, 2), '\nwhich is less than 4. This means we can assume that the population variances are equal.')

In [ ]:
stats.ttest_ind(a=dtf_2018['Downtime'], b=dtf_2019["Downtime"], equal_var=True)

Because the p-value of our test is lower than alpha = 0.05, we can reject the null hypothesis of the test. We do have sufficient evidence to say that the cycle time mean between 2018 and 2019 is different.

Let's study how the downtime behave between stations

In [ ]:
dtf.head()

So, the dtf data do not have the production line column and we will have to create one to compare the amount of downtime between lines.

In [ ]:
prod_l = cyc[['station', 'production_line']]

In [ ]:
prod_l['station'] = prod_l['station'].astype(int)
prod_l = prod_l.rename(columns = {'station':'station_no.'})

In [ ]:
dtf_pl = pd.merge(dtf, prod_l, how='left')
dtf_pl.info()

In [ ]:
sns.boxplot(data=dtf_pl, x='production_line', y='Downtime')

In [ ]:
dtf_pl.groupby(['production_line'])[['Downtime']].mean()

The means seems consistent with every station line, we can see if the mean of station 3 fall inside the intervals of station 6.

In [ ]:

dt_pl3 = dtf_pl[dtf_pl['production_line'] == 3]['Downtime']
dt_pl6 = dtf_pl[dtf_pl['production_line'] == 6]['Downtime']

In [ ]:
t_critical_3= t.ppf(0.95, len(dt_pl3))
std_error_3 = np.std(dt_pl3)/np.sqrt(len(dt_pl3))
margin_error_3 = t_critical_3 * std_error_3
lower_3 = np.mean(dt_pl3) - margin_error_3
upper_3 = np.mean(dt_pl3) - margin_error_3

t_critical_6= t.ppf(0.95, len(dt_pl6))
std_error_6 = np.std(dt_pl6)/np.sqrt(len(dt_pl6))
margin_error_6 = t_critical_6 * std_error_6
lower_6 = np.mean(dt_pl6) - margin_error_6
upper_6 = np.mean(dt_pl6) - margin_error_6


print('PRODUCTION LINE 3','\n\nThe t critical is', round(t_critical_3, 2), ', the standard error is', round(std_error_3, 2), '\nand the margin of error is', round(margin_error_3, 2), '. \nMeaning the interval of frequency are', round(np.mean(dt_pl3) - margin_error_3, 2), 'and', round(np.mean(dt_pl3) + margin_error_3, 2))

print('\n\n\nPRODUCTION LINE 6','\n\nThe t critical is', round(t_critical_6, 2), ', the standard error is', round(std_error_6, 2), '\nand the margin of error is', round(margin_error_6, 2), '. \nMeaning the interval of frequency are', round(np.mean(dt_pl6) - margin_error_6, 2), 'and', round(np.mean(dt_pl6) + margin_error_6, 2))



As we can see the production lines 3 intervals do not fall between production line 6 intervals. In fact, the others production lines do not fall in any intervals studied before. 

In [ ]:
stats.ttest_ind(a=dt_pl6, b=dt_pl3, equal_var=True)

In [ ]:
dtf_hm = dtf_pl
dtf_hm['production_line'] = dtf_hm['production_line'].astype('int')

In [ ]:
catg = { 1: 'plt_1', 2: 'plt_2', 3: 'plt_3', 4 :'plt_4', 5: 'plt_5', 6: 'plt_6', 7: 'plt_7'}
dtf_hm['production_line'] = dtf_hm['production_line'].replace(catg)
dtf_hm.head()


In [ ]:
dtf_hm2= dtf_hm.groupby('production_line')['Downtime'].apply(list)
dtf_hm2 = dtf_hm2.apply(pd.Series).T

In [ ]:
dtf_hm2.corr()

There is no correlation betweeen production line.

In [ ]:
corrmat = dtf_hm2.corr()
f, ax = plt.subplots(figsize=(12, 9))
hm = sns.heatmap(round(corrmat,2), annot=True, ax=ax, cmap="coolwarm",fmt='.2f',
                 linewidths=.05)
f.subplots_adjust(top=0.93)
t= f.suptitle('Data Feature Correlation for Production line', fontsize=14)

In [ ]:
from scipy.stats import t
def interval(data, value_conf):
   print('Intervals:     ', '(',round(np.mean(data) - ((t.ppf(value_conf, len(data))) * (np.std(data)/np.sqrt(len(data)))), 4) , ',' ,round(((t.ppf(value_conf, len(data))) * (np.std(data)/np.sqrt(len(data)))) + np.mean(data), 4),')')

In [ ]:
interval(dtf_hm2['plt_1'], 0.95)
interval(dtf_hm2['plt_2'], 0.95)
interval(dtf_hm2['plt_3'], 0.95)
interval(dtf_hm2['plt_4'], 0.95)
interval(dtf_hm2['plt_5'], 0.95)
interval(dtf_hm2['plt_6'], 0.95)
interval(dtf_hm2['plt_7'], 0.95)


In [ ]:
sns.pairplot(dtf_hm2,  palette='Set2')
plt.show()


The pearplot show there is no strong correlation between station. 

## For this project, the main production line to focus on is number 5; let's study this line and see the findings.

In [ ]:
pl5 = dtf_pl[dtf_pl['production_line'] == 'plt_5']

In [ ]:
sns.barplot(data= pl5, x='station_no.', y='Downtime')

In [ ]:
sns.boxplot(data= pl5, x='station_no.', y='Downtime')

In [ ]:
pl5_hm = pl5.groupby('station_no.')['Downtime'].apply(list)
pl5_hm = pl5_hm.apply(pd.Series).T

In [ ]:
corrmat = pl5_hm.corr()
f, ax = plt.subplots(figsize=(12, 9))
hm = sns.heatmap(round(corrmat,2), annot=True, ax=ax, cmap="coolwarm",fmt='.2f',
                 linewidths=.05)
f.subplots_adjust(top=0.93)
t= f.suptitle('Data Feature Correlation for Production line 5', fontsize=14)

**Moderately Strong Correlation** station 26 and 32: Passes Diff (.79)

In [ ]:
#Plot relationship between Poss Time Diff and Passes Diff
sns.jointplot(x= 26, y= 32, data=pl5_hm, kind="reg")

**Moderately Strong Correlation** station 24 and 26: Passes Diff (.75)

In [ ]:
sns.jointplot(x= 24, y= 26, data=pl5_hm, kind="reg")

The EDA analysis from the variables and correlations revealed that the strongest positive correlation is between station 26 and station 32 (r=0.79), which makes sense intuitively. Typically, when station 24 goes down, this does not supply specifics units that station 26 needs, causing it to go down. The same situation happens with stations 24 and station 26.